In [1]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations
import copy

import bo_methods_lib
# from bo_methods_lib.bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pympler
import pickle

from pympler import asizeof
import gpflow
import tensorflow_probability as tfp

from matplotlib import pyplot as plt

c:\Users\16822\miniconda3\envs\tests\Lib\site-packages\gpflow\versions.py:1: UserWarning: Module bo_methods_lib was already imported from None, but c:\users\16822\research_project_fall_2024\gpbo_emulators is being added to sys.path
  import pkg_resources


In [2]:
#Set Parameters
cs_name_val = 1
ep0 = 1 #Set initial ep as an even mix between exploration and exploitation
ep_enum_val = 1
meth_name_val = 8 
sep_fact = 1.0
gen_heat_map_data = True
normalize = True
noise_mean = 0
noise_std = None
kernel_enum_val = 1
lenscl = None #list([0.136113749, 221.573761, 830.968019, 1.67917241, 0.3, 0.2])
outputscl = None
retrain_GP = 1
reoptimize_obj = 1
bo_iter_tot = 1
bo_run_total = 1
runs_per_job_max = 5
bo_runs_in_job = 5
save_data = False
ei_tol = 1e-7
obj_tol = 1e-7
num_x_data = 5
gen_meth_theta = 1 
gen_meth_x = 2
gen_meth_theta_val = 2
num_val_pts = 0
num_theta_multiplier = 10#How many simulation data points to generate is equal to num_theta_multiplier*number of parameters
seed = 1

In [3]:
#Define method, ep_enum classes, indecies to consider, and kernel
meth_name = Method_name_enum(meth_name_val)
method = GPBO_Methods(meth_name)
ep_enum = Ep_enum(ep_enum_val)
kernel = Kernel_enum(kernel_enum_val)

#Define Simulator Class (Export your Simulator Object Here)
simulator = simulator_helper_test_fxns(cs_name_val, noise_mean, noise_std, seed)

#Generate Exp Data (OR Add your own experimental data as a Data class object)
set_seed = 1 #Set set_seed to 1 for data generation
gen_meth_x = Gen_meth_enum(gen_meth_x)
exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x, set_seed)

#Set simulator noise_std artifically as 5% of y_exp mean (So that noise will be set rather than trained)
simulator.noise_std = np.abs(np.mean(exp_data.y_vals))*0.05

#Create Exploration Bias Class
if ep_enum.value == 1:
    #Constant value stays constant
    ep_bias = Exploration_Bias(ep0, None, ep_enum, None, None, None, None, None, None, None)
elif ep_enum.value == 2:
    #For decay method, decay from mixed to full exploitation (alpha of 0.5) for this example
    ep_bias = Exploration_Bias(ep0, None, ep_enum, None, bo_iter_tot, None, 0.5, None, None, None)
elif ep_enum.value == 3:
    #Set ep multiplier to 1.5 as recommended in Boyle
    ep_bias = Exploration_Bias(ep0, None, ep_enum, None, None, 1.5, None, None, None, None)
elif ep_enum.value == 4:
    #Jasrasaria method will take care of itself
    ep_bias = Exploration_Bias(None, None, ep_enum, None, None, None, None, None, None, None)
else:
    raise Warning("Ep_enum value must be between 1 and 4!")
    
#Generate Sim (Training) Data (OR Add your own training data here as a Data class object)
num_theta_data = len(simulator.indices_to_consider)*num_theta_multiplier
gen_meth_theta = Gen_meth_enum(gen_meth_theta)
#Note at present, training data is always the same between jobs since we set the data generation seed to 1
sim_data = simulator.gen_sim_data(num_theta_data, num_x_data, gen_meth_theta, gen_meth_x, sep_fact, set_seed, False)

#Gen sse_sim_data and sse_sim_val_data
sim_sse_data = simulator.sim_data_to_sse_sim_data(method, sim_data, exp_data, sep_fact, False)

#Generate validation data if applicable. This is only useful for small (<5 Params + State Points). Otherwise this takes up too much memory
if num_val_pts > 0:
    gen_meth_theta_val = Gen_meth_enum(gen_meth_theta_val) #input is an integer (1 or 2)
    val_data = simulator.gen_sim_data(num_val_pts, num_x_data, gen_meth_theta_val, gen_meth_x, sep_fact, set_seed, True)
    val_sse_data = simulator.sim_data_to_sse_sim_data(method, val_data, exp_data, sep_fact, True)        
#Set validation data to None if not generating it
else:
    val_data = None
    val_sse_data = None
    gen_meth_theta_val = gen_meth_theta_val #Value is None
                    
#Define cs_name and cs_params class
#Signac saves all BO_Results in different folders, so they can have the same name
cs_name = "BO_Results"
cs_params = CaseStudyParameters(cs_name, ep0, sep_fact, normalize, kernel, lenscl, outputscl,
                                retrain_GP, reoptimize_obj, gen_heat_map_data, bo_iter_tot,
                                bo_runs_in_job, True, None, seed, ei_tol, obj_tol)
#Initialize driver class
driver = GPBO_Driver(cs_params, method, simulator, exp_data, sim_data, sim_sse_data, val_data, val_sse_data, None, ep_bias, gen_meth_theta)
#Get results
restart_bo_results, gp_stuff = driver.run_bo_restarts()

In [4]:
restart_bo_results[0].results_df.iloc[:,0:13]
#['Theta Min Obj', 'Theta Obj Act Cum', 'Min Obj Act', "Min Obj Act Cum"]

,BO Iter,Best Error,Exploration Bias,Theta Opt Acq,Opt Acq,Acq Obj Act,MSE Acq Act,Theta Min Obj,Min Obj GP,Min Obj Act,MSE Obj GP,MSE Obj Act,Time/Iter
0,1,4.858648,1.0,"[-1.0236432494005134, 0.0990726073481295]",41.109949,82.014728,16.402946,"[-1.0236432494005565, 0.09907261734806795]",41.085009,82.014728,8.217002,16.402946,12.0444


In [5]:
restart_bo_results[0].results_df.iloc[:,:13]

,BO Iter,Best Error,Exploration Bias,Theta Opt Acq,Opt Acq,Acq Obj Act,MSE Acq Act,Theta Min Obj,Min Obj GP,Min Obj Act,MSE Obj GP,MSE Obj Act,Time/Iter
0,1,4.858648,1.0,"[-1.0236432494005134, 0.0990726073481295]",41.109949,82.014728,16.402946,"[-1.0236432494005565, 0.09907261734806795]",41.085009,82.014728,8.217002,16.402946,12.0444
